# **<span style="color: black; font-size:1em;">PrOMMiS LCA Integration: Environmental Life Cycle Assessment Using PrOMMiS Modeling Results</span>**

## <span style="color:black; font-weight:bold"> Introduction </span> 

<span style = "color:black">
This document demonstrates the application of Life Cycle Assessment (LCA) methodology to the optimized results from the PrOMMiS model.  
This Jupyter Notebook presents an example LCA application to the West Kentucky No.13 Coal Refuse Flowsheet optimized in the PrOMMiS model.
</br>

<u> Goal and Scope:</u> In this Jupyter Notebook, we use life cycle assessment to evaluate the environmental impact of Rare Earth Oxide Recovery from coal mining refuse.</br>
As shown in the system boundary below, the scope of this work starts with the leaching of size-reduced REE-rich feedstock (REE: Rare Earth Elements) and ends with the recovery of mixed REO solids. The process consists of six main stages: 1) Mixing and Leaching, 2) Rougher Solvent Extraction, 3) Cleaner Solvent Extraction, 4) Precipitation, 5) Solid-Liquid (S/L) separation, and 6) Roasting. </br>
It's crucial to note here that the current script does not account for:

* Upstream processes leading to the production of REE-rich feedstock 


* Downstream processes leading to the separation of REE contained in the REO    

<u> Main Product:</u> The main product of the flowsheet modeled in this Jupyter Notebook is Rare Earth Oxide (REO) solid.

<u> Co-products or by-products:</u> None

<u> Functional Unit:</u> The function unit is 1 kg of recovered REO solids

<u> Allocation: </u> The evaluated flowsheet produces a single product with no intermediate co-products or by-product. As a result, there is no need for allocation in this modeling exercise.

</span>

<span style = "color:black">

**Useful notes, definitions, and links:**
- **PrOMMiS model:** Process Optimization and Modeling for Minerals Sustainability initiative, led by the U.S. Department of Energy (DOE), specifically under NETL (National Energy Technology Laboratory). The PrOMMiS source code can be accessed on github through the following repository link: [https://github.com/prommis/prommis/tree/main](https://github.com/prommis/prommis/tree/main)<br> PrOMMiS serves to optimize processing flowsheets. This Jupyter Notebook focuses on a flowsheet for the extraction of REE from coal refuse. 


- **UKy Flowsheet Code:** [https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py](https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py)

</span>

<div style="text-align: center;">
    <img src="images/system_boundary_1.png" width="1000"/>
</div>

## <span style="color:black; font-weight:bold"> Step 1: Import the necessary tools </span> 

In [2]:
# import main libraries
import pandas as pd
import olca_schema as olca

from netlolca.NetlOlca import NetlOlca

import prommis.uky.uky_flowsheet as uky # module to run the PrOMMiS model
import src as lca_prommis

## <span style="color:black; font-weight:bold"> Step 2: Run PrOMMiS Optimization Model for the UKy flowsheet and extract model results  </span> 

In [ ]:
# Run UKy flowsheet, and get prommis model
m, _ = uky.main() 

# Extract LCA data
prommis_data = lca_prommis.data_lca.get_lca_df(m)
prommis_data.to_csv("output/lca_df.csv")

## <span style="color:black; font-weight:bold"> Step 3: Review PrOMMiS Results </span> 

In [13]:
prommis_data.head()

,Flow,Source,In/Out,Category,Value 1,Unit 1,Value 2,Unit 2
0,Inerts,Solid Feed,In,Solid Input,22.68,kg/hr,0.6952,mass fraction
1,Scandium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000028,mass fraction
2,Yttrium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000033,mass fraction
3,Lanthanum Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000068,mass fraction
4,Cerium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000156,mass fraction


## <span style="color:black; font-weight:bold"> Step 4: Organize, Categorize, and Covert PrOMMiS Results to LCA Relevant Flows </span> 

In [4]:
# Hours is the time period for the conversion
# mol_to_kg is a boolean that indicates whether to convert moles to kg
# water_unit is the unit of water (m3 or L recommended, kg is also an option)
df = lca_prommis.convert_lca.convert_flows_to_lca_units(prommis_data, hours=1, mol_to_kg=True, water_unit='m3')
df.to_csv('output/lca_df_converted.csv', index=False)

# Finalize the LCA data
df = pd.read_csv('output/lca_df_converted.csv')
df = lca_prommis.final_lca.main()
df.to_csv('output/lca_df_finalized.csv', index=False)

# This will create a new datafrane and csv file that contains the initial prommis_data df with two additional columns:
# 'LCA Amount' and 'LCA Unit'

WARNING Dynamically registering the following numeric type:
    pymatgen.core.units.FloatWithUnit
Dynamic registration is supported for convenience, but there are known
limitations to this approach.  We recommend explicitly registering
numeric types using RegisterNumericType() or RegisterIntegerType().
Applied functional unit conversion with scaling factor: 1.5965288953841737e-05
Reference flow: 99.85% REO Product from Roaster Product
INFO Returning C:\Users\franc\AppData\Local\fedelemflowlist\FedElemFlowListMaster_v1.3.0_a79846d.parquet
Summary:
  total_flows: 17
  input_flows: 10
  output_flows: 7
  reference_products: 1
  unique_flow_types: 9
  total_lca_amount: 26057242.59062692

Flow Type Breakdown:
  Chemicals: 6
  Emissions to air: 4
  Solid Input: 1
  Solid Output: 1
  Electricity: 1
  Heat: 1
  Solid Waste: 1
  Wastewater: 1
  Water: 1


## <span style="color:black; font-weight:bold"> Step 5: Normalize Flows to the Selected Functional Unit and Review Final LCA Flows </span> 

In [5]:
# The below code will normalize the converted prommis data to a functional unit
# Note: This code is developed for the UKy flowsheet and the functional unit is automatically set to 1 kg of REO (combination of all REEs)
# TODO: This code should be developed for the other flowsheets. To achieve this, the user should be able to specify the desired functional unit
df = pd.read_csv('output/lca_df_converted.csv')

# Run the merge_flows function for the feed
REO_list = [
    "Yttrium Oxide",
    "Lanthanum Oxide",
    "Cerium Oxide",
    "Praseodymium Oxide",
    "Neodymium Oxide",
    "Samarium Oxide",
    "Gadolinium Oxide",
    "Dysprosium Oxide",
]
df = lca_prommis.final_lca.merge_flows(df, merge_source='Solid Feed', new_flow_name='374 ppm REO Feed', value_2_merge=REO_list)
# This 374 ppm value is directly calculated from the flowsheet. The original study actually used 357 ppm as the feed concentration.

# Run the merge_flows function for the product
df = lca_prommis.final_lca.merge_flows(df, merge_source='Roaster Product', new_flow_name='99.85% REO Product')

# Run the merge_flows function for the liquid waste flows
df = lca_prommis.final_lca.merge_flows(df, merge_source='Wastewater', new_flow_name='Wastewater', merge_column='Category') 
# Note: some of these streams are organic waste, but they're treated as wastewater

# Run the merge_flows function for the solid waste flows
df = lca_prommis.final_lca.merge_flows(df, merge_source='Solid Waste', new_flow_name='Solid Waste', merge_column='Category') 

# Run the finalize_df function
try:
    finalized_df = lca_prommis.final_lca.finalize_df(
        df=df, 
        reference_flow='99.85% REO Product', 
        reference_source='Roaster Product',
        water_type='raw fresh water'
    )
    
    # Get summary
    summary = lca_prommis.final_lca.get_finalize_summary(finalized_df)
    print("Summary:")
    for key, value in summary.items():
        if key != 'flow_type_breakdown':
            print(f"  {key}: {value}")
    
    print("\nFlow Type Breakdown:")
    for flow_type, count in summary['flow_type_breakdown'].items():
        print(f"  {flow_type}: {count}")
        
except Exception as e:
    print(f"Error during finalization: {e}")

finalized_df.to_csv('output/lca_df_finalized.csv', index=False)
finalized_df.head(10)

Applied functional unit conversion with scaling factor: 1.5965288953841737e-05
Reference flow: 99.85% REO Product from Roaster Product
INFO Returning C:\Users\franc\AppData\Local\fedelemflowlist\FedElemFlowListMaster_v1.3.0_a79846d.parquet
Summary:
  total_flows: 17
  input_flows: 10
  output_flows: 7
  reference_products: 1
  unique_flow_types: 9
  total_lca_amount: 26057242.59062692

Flow Type Breakdown:
  Chemicals: 6
  Emissions to air: 4
  Solid Input: 1
  Solid Output: 1
  Electricity: 1
  Heat: 1
  Solid Waste: 1
  Wastewater: 1
  Water: 1


,Flow_Name,LCA_Amount,LCA_Unit,Is_Input,Reference_Product,Flow_Type,Category,Context,UUID,Description
0,374 ppm REO Feed,1.420741e+06,kg,True,False,Solid Input,Technosphere flows,,,
1,99.85% REO Product,1.000000e+00,kg,False,True,Solid Output,Technosphere flows,,,
2,Carbon dioxide,9.283150e-01,kg,False,False,Emissions to air,Elementary flows,emission/air,b6f010fb-a764-3063-af2d-bcb8309a97b7,
3,DEHPA,7.580114e+05,kg,True,False,Chemicals,Technosphere flows,,,
4,"Electricity, AC, 120 V",1.628458e+03,kWh,True,False,Electricity,Technosphere flows,,,
5,Heat,1.629930e+05,MJ,True,False,Heat,Technosphere flows,,,
6,Hydrochloric Acid,3.754395e+03,kg,True,False,Chemicals,Technosphere flows,,,
7,Kerosene,6.369844e+05,kg,True,False,Chemicals,Technosphere flows,,,
8,Nitrogen,1.020383e+01,kg,False,False,Emissions to air,Elementary flows,emission/air,b11bd361-08c0-33b4-b624-2baae45ed7d8,
9,Oxalic Acid,3.034436e+04,kg,True,False,Chemicals,Technosphere flows,,,


## <span style="color:black; font-weight:bold"> Step 6: Connect to openLCA </span> 

#### <span style="color:black; font-weight:bold"> Using this Jupyter Notebook with openLCA </span> 

<span style = "color:black">
In the following modeling stages, this Jupyter Notebook creates an openLCA process, converts it to a product system, and calculates the environmental impacts of the flowsheet described in the introduction.  
As a result, please follow these steps before proceeding to the next cell in this notebook:

* Open openLCA on your desktop, prefereable with a version > 2.50

* Download the PrOMMiS openLCA database from EDX (Next Cell). The PrOMMiS database contains all the processes and impact assessment methods needed to build and evaluate the UKy flowsheet in openLCA.

* Import and open the PrOMMiS openLCA database

* Connect to IPC server (Port 8080). Go to Tools > Developer Tools > IPC Server > Connect

</span>
 

#### <span style="color:black; font-weight:bold"> Download PrOMMiS Database from EDX </span> 

<span style = "color:black">
The cell below downloads the PrOMMiS openLCA database from EDX in JSONLD format and save it in a dedicated 'resources' folder in the current working directory. The following cell will require your EDX API key to download the requested database. Note that the resource ID is set in the next cell but it's advised to cross-check that the ID is correct. The database is constantly being improved and updated, and as a result can have a different resource ID.  

To verify that you're using the correct resource ID, follow this link: [https:/edx.netl.doe.gov/workspace/resources/prommis-lca-integration](https:/edx.netl.doe.gov/workspace/resources/prommis-lca-integration) 
</span>
 

In [ ]:
# This is the resource ID of the PrOMMiS openLCA database as of September 17, 2025
resource_id = '07e09dfa-1928-46f0-8ced-56d0010b056f'

# The function below downloads the database and saves it in a 'resources' folder in the current work directory
lca_prommis.import_db.import_db(resource_id)

#### <span style="color:black; font-weight:bold"> Connect to openLCA IPC Server </span> 

In [7]:
netl = NetlOlca()
netl.connect()
netl.read()

INFO Connected on http://localhost:8080
INFO Read UUIDs from IPC connection.


#### <span style="color:black; font-weight:bold"> Import Exchanges Table </span> 

<span style="color:black"> 

</span>

In [ ]:
# Save the LCA data produced in previous steps in a dataframe called df
df = pd.read_csv('output/lca_df_finalized.csv')
df.head(5)

,Flow_Name,LCA_Amount,LCA_Unit,Is_Input,Reference_Product,Flow_Type,Category,Context,UUID,Description
0,374 ppm REO Feed,1.420741e+06,kg,True,False,Solid Input,Technosphere flows,NaN,NaN,NaN
1,99.85% REO Product,1.000000e+00,kg,False,True,Solid Output,Technosphere flows,NaN,NaN,NaN
2,Carbon dioxide,9.283150e-01,kg,False,False,Emissions to air,Elementary flows,emission/air,b6f010fb-a764-3063-af2d-bcb8309a97b7,NaN
3,DEHPA,7.580114e+05,kg,True,False,Chemicals,Technosphere flows,NaN,NaN,NaN
4,"Electricity, AC, 120 V",1.628458e+03,kWh,True,False,Electricity,Technosphere flows,NaN,NaN,NaN
5,Heat,1.629930e+05,MJ,True,False,Heat,Technosphere flows,NaN,NaN,NaN
6,Hydrochloric Acid,3.754395e+03,kg,True,False,Chemicals,Technosphere flows,NaN,NaN,NaN
7,Kerosene,6.369844e+05,kg,True,False,Chemicals,Technosphere flows,NaN,NaN,NaN
8,Nitrogen,1.020383e+01,kg,False,False,Emissions to air,Elementary flows,emission/air,b11bd361-08c0-33b4-b624-2baae45ed7d8,NaN
9,Oxalic Acid,3.034436e+04,kg,True,False,Chemicals,Technosphere flows,NaN,NaN,NaN


#### <span style="color:black; font-weight:bold"> Enter Process Information </span> 

##### <span style="color:black; font-weight:bold"> Unit Process Name </span> 

In [9]:
process_name = "Test Process 4"  # Replace with the actual process name you want to use

##### <span style="color:black; font-weight:bold"> Unit Process Description </span> 

In [10]:
process_description = " Example Process Description"  # Replace with the actual process description you want to use

##### <span style="color:black; font-weight:bold"> Create process with exchanges </span> 

<span style="color:black">  

The functioning features include:
* Create a new empty process
* Create an exchange of the reference flow

Intermediate functions/features that have been developed include:
* Search for a flow by keyword
* Search for processes that produce a given flow
    * This feature is being optimized to minimize user waiting time
* Create exchanges for elementary flows given their uuid, amount, and unit

The features under development:
* Create exchange for a given flow and process IDs
* Develop an interactive user interface, allowing the user to create a new process in openLCA
</span>

In [11]:
# create process usign the create_new_process function
process = lca_prommis.create_lca.create_new_process(netl,df,process_name,process_description)
# get the process uuid to be used in the following step when create a product system
process_id = netl.query(olca.Process, process.id)
# TODO: give option for the user to enter another keyword when searching for a flow
# NOTE: For future discussion - is it better to select the flow then the provider (current status) or pick provider first then select relevant flows?

Creating exchange database, this may take a couple minutes...


Enter flow name keyword(s). Type 'skip' to skip this flow. Press enter to use 374 ppm REO Feed:  skip


Skipping flow: 374 ppm REO Feed
Do you want to select an existing quantitative reference flow or create a new one?
1. Select existing flow
2. Create new flow


Enter your choice (1 or 2):  1
Enter flow name keyword(s). Type 'skip' to skip this flow.  copper


Searching for flows containing 'copper'...
Found 196 flows matching 'copper'
Filtered to 9 PRODUCT_FLOW flows

Matching flows:
--------------------------------------------------------------------------------
  1. copper
     UUID: 8e7135e4-1274-9414-a155-9dc1e2fd9bb4

  2. Copper, primary, at refinery
     UUID: 018c2645-7622-321b-a9ef-760b1fd90371

  3. Copper oxide, at plant
     UUID: bc9f2a96-56a5-4219-8c61-58356a0eaabd

  4. Crude cobalt hydroxide; at plant; 35% cobalt and 0.47% copper
     UUID: 2b274bb6-ed91-4ec0-8386-07c602bd6eb9

  5. Copper, at regional storage
     UUID: 19fb905b-a259-3480-848f-af6a058a4d99

  6. Copper product manufacturing, average metal working
     UUID: 24744899-a338-3943-9f83-764d10600319

  7. Copper Chromium Arsenate (CCA), at plant
     UUID: 8d2dd2ca-07cb-3ace-b1db-6f7efff79337

  8. Copper, at plant
     UUID: 12e1296e-4e80-35e8-910a-de17402a0b9c

  9. Cobalt ore; at mine; 0.47% cobalt and 2.44% copper
     UUID: 4724dc20-fc73-4f23-83f7-12ac98f37e

...

Skipping flow: Solid Waste


Enter flow name keyword(s). Type 'skip' to skip this flow. Press enter to use Sulfuric Acid:  skip


Skipping flow: Sulfuric Acid


Enter flow name keyword(s). Type 'skip' to skip this flow. Press enter to use Wastewater:  skip


Skipping flow: Wastewater
Successfully created process: Test Process 4
Process saved successfully to openLCA database!


##### <span style="color:black; font-weight:bold"> Create product system </span> 

In [ ]:
# create product system
ps = lca_prommis.create_ps(netl, process_id)
# get product system id to use in next step 
ps_uuid = netl.query(olca.ProductSystem, ps.id)

##### <span style="color:black; font-weight:bold"> Run analysis </span> 

In [ ]:
# First we need to select the impact assessment method uuid
# In this project we are developing our own impact assessment 
# method that calculates the global warming potenit, cumulative energy demand, water consumption, and acidification potential
# The developed method is included in the database attached with this jupyter notebook
# TODO: add uuid of impact assessment method 
impact_method_uuid = '60cb71ff-0ef0-4e6c-9ce7-c885d921dd15'

result = lca_prommis.run_analysis(netl, ps_uuid, impact_method_uuid)

##### <span style="color:black; font-weight:bold"> Generate Results - Total Environmental Impact </span> 

In [ ]:
# use the generate_total_results function to generate the total environmental impacts
# this also stores the results in a csv stored in the output folder
total_impacts = lca_prommis.generate_total_results(result)
total_impacts

##### <span style="color:black; font-weight:bold"> Generate Results - Contribution Tree </span> 

In [ ]:
# use the generate_contribution_tree to generate the results by category
# here we have to determine the number of nodes and levels in the generates tree
# the number of nodes reflects the number of child/constituting nodes to be expanded (e.g., electricity, heat, sulfuric acic, etc.)
# selecting three nodes will show the top three contributors only
max_expand_nodes = 5 # this is an example and the user can change this
# the number of levels reflects the number of steps away from the main product
max_expand_levels = 2
# Note: this step will store the results for each impact cateogory in a separate excel file
lca_prommis.generate_contribution_tree(netl, max_expand_levels, max_expand_nodes)